# 설명

**한국청소년활동진흥원(KYWA)**

"2024년 청소년데이터 분석·활용 공모전"에 사용한 데이터를 가공하고 추출하는 과정

팀명 - 수틀리면 구운족발

팀원 - 정우정(팀장), 김택균, 이은주

데이터 가공 - 정우정

# 사용한 데이터

KOSIS, 공공데이터포털 등을 토대로 추출한 데이터들, 사전에 일부 전처리를 진행

* 지역별 인구수.csv
* 지역별 10대 인구비율.csv
* 지역별 청소년증 발급.csv
* 지정문화재 수.csv
* 국가등록 문화재 수.csv
* 지역별도서관현황.csv
* 연간 대출 도서 수.csv
* 도서관방문자수.csv
* 청소년 수련 시설 수.csv
* 사회복지시설 수.csv
* 대중교통이용비율(시내버스).csv
* 대중교통이용비율(지하철).csv
* 대중교통이용횟수( 주 1~5).csv
* 대중교통이용횟수( 주 6~10).csv
* 대중교통이용횟수( 주 11~15).csv
* 대중교통이용횟수( 주 16~20).csv
* 대중교통이용횟수( 주 21~).csv
* 매출금액_by_year.csv
* 예매건수_by_year.csv
* 시설수_by_year.csv
* 공연장수_by_year.csv
* 공개영화수_by_year.csv
* 연도별 궁능 관람객 수.csv
* 청소년증 발급 비율.csv
* 지역별 도서관 현황.csv
* 연간 대출 도서 수.csv
* 도서관방문자수.csv

# 생성한 데이터

* 지역별 10대 인구수.csv
* 청소년증 발급 비율.csv
* 1차 데이터셋.csv
* 분석 최종 데이터셋.csv
* {시도별}_data.csv

# 데이터 처리

라이브러리 호출

In [1]:
import pandas as pd

## 1. 지역별 10대 인구수 데이터 생성

지역별 인구수 데이터와 지역별 10대 인구비율을 추출하여, 지역별 10대 인구수 데이터 생성

In [2]:
# 데이터 불러오기

# 지역별 전체 인구수 데이터 로드
population_data = pd.read_csv('/content/지역별 인구수.csv')

# 지역별 10대 인구 비율(%) 데이터 로드
teen_rate_data = pd.read_csv('/content/지역별 10대 인구비율.csv')

In [3]:
# 불필요한 컬럼 제거

# CSV 저장 과정에서 생긴 Unnamed 컬럼 제거
teen_rate_data = teen_rate_data.loc[:, ~teen_rate_data.columns.str.contains('Unnamed')]

In [4]:
# 지역별 10대 인구수 계산

# 인구수 × (10대 비율 / 100) 로 계산
teen_population = population_data.set_index('시도별').mul(teen_rate_data.set_index('시도별') / 100)

In [5]:
# 소수점 반올림 처리
teen_population = teen_population.round(0)

In [6]:
# 데이터 저장
teen_population.reset_index(inplace=True)

output_file_path = '/content/지역별 10대 인구수.csv'
teen_population.to_csv(output_file_path, index=False)

In [7]:
# 계산된 데이터의 상위 5개 행 출력
print(teen_population.head())

  시도별       2004       2005       2006       2007       2008       2009  \
0  서울  1266503.0  1246559.0  1242086.0  1238433.0  1230616.0  1219706.0   
1  부산   489589.0   481817.0   475662.0   469851.0   462837.0   454286.0   
2  대구   361496.0   361320.0   360559.0   360609.0   360239.0   357615.0   
3  인천   374887.0   378176.0   381360.0   383749.0   384260.0   381200.0   
4  광주   220603.0   221289.0   223630.0   227047.0   230488.0   232965.0   

        2010       2011       2012       2013       2014      2015      2016  \
0  1198527.0  1172408.0  1128077.0  1076923.0  1029437.0  976222.0  923227.0   
1   444665.0   433848.0   415080.0   395336.0   376302.0  355238.0  335045.0   
2   353617.0   348878.0   336603.0   322826.0   308868.0  293041.0  275662.0   
3   378768.0   374369.0   362909.0   348608.0   335741.0  321151.0  307599.0   
4   234052.0   233419.0   227067.0   219410.0   211093.0  201189.0  189869.0   

       2017      2018      2019      2020      2021      2022  
0  8

## 2. 청소년증 발급 비율 데이터 생성

위에서 생성한 지역별 10대 인구수 데이터와 지역별 청소년증 발급 데이터를 통해 청소년증 발급 비율 데이터 생성

In [8]:
# 데이터 불러오기
teen_population_data = pd.read_csv('/content/지역별 10대 인구수.csv', encoding='utf-8')
id_issuance_data = pd.read_csv('/content/지역별 청소년증 발급.csv')

In [9]:
# 발급 데이터 숫자 변환
id_issuance_data.replace(',', '', regex=True, inplace=True)
id_issuance_data.iloc[:, 1:] = id_issuance_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

In [10]:
# 10대 인구 데이터 구조 변환
teen_population_melted = teen_population_data.melt(id_vars=['시도별'], var_name='연도', value_name='10대 인구수')
teen_population_melted['연도'] = teen_population_melted['연도'].astype(int)

In [11]:
# 발급 데이터 구조 변환
id_issuance_melted = id_issuance_data.melt(id_vars=['시도별'], var_name='연도', value_name='발급 인원')
id_issuance_melted['연도'] = id_issuance_melted['연도'].astype(int)

In [12]:
# 두 데이터 병합
merged_data = pd.merge(teen_population_melted, id_issuance_melted, on=['시도별', '연도'])

In [13]:
# 청소년증 발급 비율 계산
merged_data['청소년증 발급 비율'] = (
    merged_data['발급 인원'] / merged_data['10대 인구수'] * 100
).map(lambda x: '{:.3f}'.format(x))

In [14]:
# 결과 저장
output_file_path = '/content/청소년증_발급_비율.csv'
merged_data.to_csv(output_file_path, index=False)

print(f'결과 파일이 저장된 경로: {output_file_path}')

결과 파일이 저장된 경로: /content/청소년증_발급_비율.csv


## 3. 1차 데이터셋 생성

지역별 인구수, 문화재 현황, 도서관 이용 데이터를 통합하여 인구 10만 명당 문화·도서관 관련 지표를 산출한 1차 데이터셋 생성 코드

In [15]:
# 데이터 경로 설정 및 파일 불러오기
file_paths = {
    "population": '/content/지역별 인구수.csv',
    "heritage_designated": '/content/지정문화재 수.csv',
    "heritage_registered": '/content/국가등록 문화재 수.csv',
    "library": '/content/지역별도서관현황.csv',
    "loan": '/content/연간 대출 도서 수.csv',
    "visitor": "/content/도서관방문자수.csv"
}

data_population         = pd.read_csv(file_paths["population"], encoding='utf-8')
data_heritage_designated = pd.read_csv(file_paths["heritage_designated"], encoding='EUC-KR')
data_heritage_registered = pd.read_csv(file_paths["heritage_registered"], encoding='UTF-8-SIG')
data_library            = pd.read_csv(file_paths["library"], encoding='cp949')
data_loan               = pd.read_csv(file_paths["loan"], encoding='cp949')
data_visitor            = pd.read_csv(file_paths["visitor"], encoding='cp949')

In [16]:
# 데이터 정규화 -> 연도별 정리
def melt_data(df, value_name):
    melted = df.melt(id_vars=["시도별"], var_name="연도", value_name=value_name)
    melted["연도"] = melted["연도"].astype(int)
    return melted

population_m = melt_data(data_population, "인구수")
heritage_des_m = melt_data(data_heritage_designated, "지정문화재 수")
heritage_reg_m = melt_data(data_heritage_registered, "국가등록 문화재 수")
library_m = melt_data(data_library, "도서관 수")
loan_m = melt_data(data_loan, "연간대출 도서 수")
visitor_m = melt_data(data_visitor, "도서관 방문자 수")

In [17]:
# 데이터 병합 (시도별 + 연도 기준)
merged = (
    heritage_des_m
    .merge(heritage_reg_m, on=["시도별", "연도"])
    .merge(population_m, on=["시도별", "연도"])
    .merge(library_m, on=["시도별", "연도"])
    .merge(loan_m, on=["시도별", "연도"])
    .merge(visitor_m, on=["시도별", "연도"])
)

In [18]:
# 인구 10만 명당 지표 계산
merged["지정문화재 수 (10만명당)"] = merged["지정문화재 수"] / merged["인구수"] * 100000
merged["국가등록 문화재 수 (10만명당)"] = merged["국가등록 문화재 수"] / merged["인구수"] * 100000
merged["도서관 수 (10만명당)"] = merged["도서관 수"] / merged["인구수"] * 100000
merged["연간대출 도서 수 (10만명당)"] = merged["연간대출 도서 수"] / merged["인구수"] * 100000
merged["도서관 방문자 수 (10만명당)"] = merged["도서관 방문자 수"] / merged["인구수"] * 100000


In [19]:
# 필요한 열만 정리하여 저장
result = merged[
    ["시도별", "연도",
     "지정문화재 수 (10만명당)",
     "국가등록 문화재 수 (10만명당)",
     "도서관 수 (10만명당)",
     "연간대출 도서 수 (10만명당)",
     "도서관 방문자 수 (10만명당)"]
]

output_path = '/content/1차 데이터셋.csv'
result.to_csv(output_path, index=False, encoding='utf-8-sig')

In [20]:
# 결과 확인
output_path

'/content/1차 데이터셋.csv'

In [21]:
import pandas as pd

# ==========================================
# 1. 데이터 경로 설정 및 파일 불러오기
# ==========================================
file_paths = {
    "population": '/content/지역별 인구수.csv',
    "heritage_designated": '/content/지정문화재 수.csv',
    "heritage_registered": '/content/국가등록 문화재 수.csv',
    "library": '/content/지역별도서관현황.csv',
    "loan": '/content/연간 대출 도서 수.csv',
    "visitor": "/content/도서관방문자수.csv"
}

data_population         = pd.read_csv(file_paths["population"], encoding='utf-8')
data_heritage_designated = pd.read_csv(file_paths["heritage_designated"], encoding='EUC-KR')
data_heritage_registered = pd.read_csv(file_paths["heritage_registered"], encoding='UTF-8-SIG')
data_library            = pd.read_csv(file_paths["library"], encoding='cp949')
data_loan               = pd.read_csv(file_paths["loan"], encoding='cp949')
data_visitor            = pd.read_csv(file_paths["visitor"], encoding='cp949')


# ==========================================
# 2. 데이터 정규화(melt) → 연도별 tidy 형태로 정리
# ==========================================
def melt_data(df, value_name):
    melted = df.melt(id_vars=["시도별"], var_name="연도", value_name=value_name)
    melted["연도"] = melted["연도"].astype(int)
    return melted

population_m = melt_data(data_population, "인구수")
heritage_des_m = melt_data(data_heritage_designated, "지정문화재 수")
heritage_reg_m = melt_data(data_heritage_registered, "국가등록 문화재 수")
library_m = melt_data(data_library, "도서관 수")
loan_m = melt_data(data_loan, "연간대출 도서 수")
visitor_m = melt_data(data_visitor, "도서관 방문자 수")


# ==========================================
# 3. 데이터 병합 (시도별 + 연도 기준)
# ==========================================
merged = (
    heritage_des_m
    .merge(heritage_reg_m, on=["시도별", "연도"])
    .merge(population_m, on=["시도별", "연도"])
    .merge(library_m, on=["시도별", "연도"])
    .merge(loan_m, on=["시도별", "연도"])
    .merge(visitor_m, on=["시도별", "연도"])
)


# ==========================================
# 4. 인구 10만 명당 지표 계산
# ==========================================
merged["지정문화재 수 (10만명당)"] = merged["지정문화재 수"] / merged["인구수"] * 100000
merged["국가등록 문화재 수 (10만명당)"] = merged["국가등록 문화재 수"] / merged["인구수"] * 100000
merged["도서관 수 (10만명당)"] = merged["도서관 수"] / merged["인구수"] * 100000
merged["연간대출 도서 수 (10만명당)"] = merged["연간대출 도서 수"] / merged["인구수"] * 100000
merged["도서관 방문자 수 (10만명당)"] = merged["도서관 방문자 수"] / merged["인구수"] * 100000


# ==========================================
# 5. 필요한 열만 정리하여 저장
# ==========================================
result = merged[
    ["시도별", "연도",
     "지정문화재 수 (10만명당)",
     "국가등록 문화재 수 (10만명당)",
     "도서관 수 (10만명당)",
     "연간대출 도서 수 (10만명당)",
     "도서관 방문자 수 (10만명당)"]
]

output_path = '/content/1차 데이터셋.csv'
result.to_csv(output_path, index=False, encoding='utf-8-sig')

output_path


'/content/1차 데이터셋.csv'

## 4. 분석 최종 데이터셋 생성

지역별 공공·문화·교통·청소년 관련 데이터를 통합하고, 인구 10만 명당 표준화 지표로 변환하여 최종 분석용 데이터셋을 생성하는 코드

In [22]:
  # 데이터 불러오기
  file_paths_updated = {
      "지역별 인구수": '/content/지역별 인구수.csv',
      "지정문화재 수": '/content/지정문화재 수.csv',
      "국가등록 문화재 수": '/content/국가등록 문화재 수.csv',
    "청소년 수련 시설 수": '/content/청소년 수련 시설 수.csv',
      "지역별 10대 인구수": '/content/지역별 10대 인구수.csv',
      "사회 복지 시설 수": '/content/사회복지시설 수.csv',
      "문화시설 수": '/content/인구 대비 문화시설 수.csv',
      "대중교통이용-시내버스":"/content/대중교통이용비율(시내버스).csv",
      "대중교통이용-지하철":"/content/대중교통이용비율(지하철).csv",
      "대중교통이용 횟수(1~5)":"/content/대중교통이용횟수(주 1~5).csv",
      "대중교통이용 횟수(6~10)":"/content/대중교통이용횟수(주 6~10).csv",
      "대중교통이용 횟수(11~15)":"/content/대중교통이용횟수(주 11~15).csv",
      "대중교통이용 횟수(16~20)":"/content/대중교통이용횟수(주 16~20).csv",
      "대중교통이용 횟수(21~)":"/content/대중교통이용횟수(주 21~).csv",
      "매출금액": "/content/매출금액_by_year.csv",
      "예매건수": "/content/예매건수_by_year.csv",
      "시설수":"/content/시설수_by_year.csv",
      "공연장수":"/content/공연장수_by_year.csv",
      "공개영화수":"/content/공개영화수_by_year.csv",
      "궁능관광객수": "/content/연도별 궁능 관람객 수.csv",
      "청소년증발급비율": "/content/청소년증 발급 비율.csv",
       "도서관 수": '/content/지역별도서관현황.csv',
    "연간대출 도서 수" : '/content/연간 대출 도서 수.csv',
    "도서관 방문자 수" : "/content/도서관방문자수.csv"

  }

In [23]:
# 파일을 각기 해당 인코딩으로 불러오기
data_population = pd.read_csv(file_paths_updated["지역별 인구수"], encoding='utf-8')
data_heritage_designated = pd.read_csv(file_paths_updated["지정문화재 수"], encoding='EUC-KR')
data_heritage_registered = pd.read_csv(file_paths_updated["국가등록 문화재 수"], encoding='UTF-8-SIG')
data_train_facilities = pd.read_csv(file_paths_updated["청소년 수련 시설 수"], encoding='utf-8')
data_population_10 = pd.read_csv(file_paths_updated["지역별 10대 인구수"], encoding='utf-8')
data_welfare_facilities = pd.read_csv(file_paths_updated["사회 복지 시설 수"], encoding='cp949')
data_cultural_facilities = pd.read_csv(file_paths_updated["문화시설 수"], encoding='cp949')
data_public_transport_bus = pd.read_csv(file_paths_updated["대중교통이용-시내버스"], encoding='utf-8')
data_public_transport_subway = pd.read_csv(file_paths_updated["대중교통이용-지하철"], encoding='utf-8')
data_public_transport_1_5 = pd.read_csv(file_paths_updated["대중교통이용 횟수(1~5)"], encoding='utf-8')
data_public_transport_6_10 = pd.read_csv(file_paths_updated["대중교통이용 횟수(6~10)"], encoding='utf-8')
data_public_transport_11_15 = pd.read_csv(file_paths_updated["대중교통이용 횟수(11~15)"], encoding='utf-8')
data_public_transport_16_20 = pd.read_csv(file_paths_updated["대중교통이용 횟수(16~20)"], encoding='utf-8')
data_public_transport_21 = pd.read_csv(file_paths_updated["대중교통이용 횟수(21~)"], encoding='utf-8')
data_sales = pd.read_csv(file_paths_updated["매출금액"], encoding='utf-8')
data_reservations = pd.read_csv(file_paths_updated["예매건수"], encoding='utf-8')
data_facilities = pd.read_csv(file_paths_updated["시설수"], encoding='utf-8')
data_theaters = pd.read_csv(file_paths_updated["공연장수"], encoding='utf-8')
data_movies = pd.read_csv(file_paths_updated["공개영화수"], encoding='utf-8')
data_royal_tombs = pd.read_csv(file_paths_updated["궁능관광객수"], encoding='cp949')
data_youth_enrollment_rate = pd.read_csv(file_paths_updated["청소년증발급비율"], encoding='utf-8')
data_library = pd.read_csv(file_paths_updated["도서관 수"], encoding='cp949')
data_loan = pd.read_csv(file_paths_updated["연간대출 도서 수"], encoding='cp949')
data_visitor = pd.read_csv(file_paths_updated["도서관 방문자 수"], encoding='cp949')

In [24]:
# 지정문화재·문화·교통·공연·도서관 지표를 '시도별-연도' 형태의 Long Format으로 변환(melt)
data_heritage_designated_melted = data_heritage_designated.melt(id_vars=["시도별"], var_name="연도", value_name="지정문화재 수")
data_heritage_registered_melted = data_heritage_registered.melt(id_vars=["시도별"], var_name="연도", value_name="국가등록 문화재 수")
data_train_facilities_melted = data_train_facilities.melt(id_vars=["시도별"], var_name="연도", value_name="청소년 수련시설 수")
data_population_melted = data_population.melt(id_vars=["시도별"], var_name="연도", value_name="인구수")
data_population_10_melted = data_population_10.melt(id_vars=["시도별"], var_name="연도", value_name="10대 인구수")
data_welfare_facilities_melted = data_welfare_facilities.melt(id_vars=["시도별"], var_name="연도", value_name="사회복지 시설 수")
data_cultural_facilities_melted = data_cultural_facilities.melt(id_vars=["시도별"], var_name="연도", value_name="문화시설 수")

data_public_transport_bus_melted = data_public_transport_bus.melt(id_vars=["시도별"], var_name="연도", value_name="시내버스이용")
data_public_transport_subway_melted = data_public_transport_subway.melt(id_vars=["시도별"], var_name="연도", value_name="지하철이용")
data_public_transport_1_5_melted = data_public_transport_1_5.melt(id_vars=["시도별"], var_name="연도", value_name="대중교통1~5회이용")
data_public_transport_6_10_melted = data_public_transport_6_10.melt(id_vars=["시도별"], var_name="연도", value_name="대중교통6~10회이용")
data_public_transport_11_15_melted = data_public_transport_11_15.melt(id_vars=["시도별"], var_name="연도", value_name="대중교통11~15회이용")
data_public_transport_16_20_melted = data_public_transport_16_20.melt(id_vars=["시도별"], var_name="연도", value_name="대중교통16~20회이용")
data_public_transport_21_melted = data_public_transport_21.melt(id_vars=["시도별"], var_name="연도", value_name="대중교통21회이상이용")

data_sales_melted = data_sales.melt(id_vars=["시도별"], var_name="연도", value_name="매출금액")
data_reservations_melted = data_reservations.melt(id_vars=["시도별"], var_name="연도", value_name="예매건수")
data_facilities_melted = data_facilities.melt(id_vars=["시도별"], var_name="연도", value_name="시설수")
data_theaters_melted = data_theaters.melt(id_vars=["시도별"], var_name="연도", value_name="공연장수")
data_movies_melted = data_movies.melt(id_vars=["시도별"], var_name="연도", value_name="공개영화수")

data_royal_tombs_melted = data_royal_tombs.melt(id_vars=["시도별"], var_name="연도", value_name="궁능관광객수")

data_library_melted = data_library.melt(id_vars=["시도별"], var_name="연도", value_name="도서관 수")
data_loan_melted = data_loan.melt(id_vars=["시도별"], var_name="연도", value_name="연간대출 도서 수")
data_visitor_melted = data_visitor.melt(id_vars=["시도별"], var_name="연도", value_name="도서관 방문자 수")


In [25]:
# 연도를 정수형으로 변환하여 데이터 간 정렬을 일치시킴
data_heritage_designated_melted["연도"] = data_heritage_designated_melted["연도"].astype(int)
data_heritage_registered_melted["연도"] = data_heritage_registered_melted["연도"].astype(int)
data_train_facilities_melted["연도"] = data_train_facilities_melted["연도"].astype(int)
data_population_melted["연도"] = data_population_melted["연도"].astype(int)
data_population_10_melted["연도"] = data_population_10_melted["연도"].astype(int)
data_welfare_facilities_melted["연도"] = data_welfare_facilities_melted["연도"].astype(int)
data_cultural_facilities_melted["연도"] = data_cultural_facilities_melted["연도"].astype(int)

data_public_transport_bus_melted["연도"] = data_public_transport_bus_melted["연도"].astype(int)
data_public_transport_subway_melted["연도"] = data_public_transport_subway_melted["연도"].astype(int)
data_public_transport_1_5_melted["연도"] = data_public_transport_1_5_melted["연도"].astype(int)
data_public_transport_6_10_melted["연도"] = data_public_transport_6_10_melted["연도"].astype(int)
data_public_transport_11_15_melted["연도"] = data_public_transport_11_15_melted["연도"].astype(int)
data_public_transport_16_20_melted["연도"] = data_public_transport_16_20_melted["연도"].astype(int)
data_public_transport_21_melted["연도"] = data_public_transport_21_melted["연도"].astype(int)

data_sales_melted["연도"] = data_sales_melted["연도"].astype(int)
data_reservations_melted["연도"] = data_reservations_melted["연도"].astype(int)
data_facilities_melted["연도"] = data_facilities_melted["연도"].astype(int)
data_theaters_melted["연도"] = data_theaters_melted["연도"].astype(int)
data_movies_melted["연도"] = data_movies_melted["연도"].astype(int)

data_royal_tombs_melted["연도"] = data_royal_tombs_melted["연도"].astype(int)
data_youth_enrollment_rate["연도"] = data_youth_enrollment_rate["연도"].astype(int)

data_library_melted["연도"] = data_library_melted["연도"].astype(int)
data_loan_melted["연도"] = data_loan_melted["연도"].astype(int)
data_visitor_melted["연도"] = data_visitor_melted["연도"].astype(int)


In [26]:
# 청소년증 발급 비율 데이터 전처리
data_youth_enrollment_rate['연도'] = pd.to_numeric(data_youth_enrollment_rate['연도'], errors='coerce')
data_youth_enrollment_rate = data_youth_enrollment_rate[["시도별","연도","청소년증 발급 비율"]]
data_youth_enrollment_rate = data_youth_enrollment_rate.rename(columns={"청소년증 발급 비율": "청소년증발급비율"})

In [27]:
# 병합을 통해 모든 데이터를 하나의 데이터프레임으로 통합
merged_data = data_heritage_designated_melted.merge(data_heritage_registered_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_population_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_train_facilities_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_population_10_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_welfare_facilities_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_cultural_facilities_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_bus_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_subway_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_1_5_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_6_10_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_11_15_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_16_20_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_public_transport_21_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_sales_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_reservations_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_facilities_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_theaters_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_movies_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_library_melted, on=["시도별", "연도"])
merged_data = merged_data.merge(data_loan_melted, on=["시도별", "연도"])
merged_data = merged_data.merge(data_visitor_melted, on=["시도별", "연도"])
merged_data = merged_data.merge(data_royal_tombs_melted, on=["시도별", "연도"], how='left')
merged_data = merged_data.merge(data_youth_enrollment_rate, on=["시도별", "연도"], how='left')

merged_data.head()

,시도별,연도,지정문화재 수,국가등록 문화재 수,인구수,청소년 수련시설 수,10대 인구수,사회복지 시설 수,문화시설 수,시내버스이용,...,매출금액,예매건수,시설수,공연장수,공개영화수,도서관 수,연간대출 도서 수,도서관 방문자 수,궁능관광객수,청소년증발급비율
0,서울,2020,1677.0,229.0,9617795.0,19,777118.0,313.0,4.4,32.1,...,9032189.0,214582.0,568.0,733.0,418.0,188,48995263,19927153,NaN,1.950
1,부산,2020,490.0,22.0,3355632.0,7,274826.0,60.0,3.9,56.3,...,229775.0,12341.0,83.0,108.0,64.0,47,4405669,5699121,NaN,3.396
2,대구,2020,273.0,13.0,2414262.0,3,228631.0,25.0,3.8,57.1,...,312941.0,18140.0,69.0,93.0,90.0,44,3962135,3701332,NaN,2.752
3,인천,2020,259.0,9.0,2951065.0,7,271203.0,86.0,4.1,50.7,...,80517.0,5012.0,36.0,49.0,31.0,55,3578558,4540958,NaN,2.876
4,광주,2020,140.0,21.0,1479550.0,1,158164.0,13.0,5.1,83.6,...,126960.0,6718.0,35.0,51.0,25.0,24,1591535,2032998,NaN,3.469


In [28]:
# 인구 10만 명당 지정문화재 수와 국가등록 문화재 수 계산
merged_data["지정문화재(10만명당)"] = (merged_data["지정문화재 수"] / merged_data["인구수"]) * 100000
merged_data["국가등록문화재(10만명당)"] = (merged_data["국가등록 문화재 수"] / merged_data["인구수"]) * 100000

# 청소년 인구 10만 명당 청소년 수련 시설 수 계산
merged_data["청소년수련시설(청소년 10만명당)"] = (merged_data["청소년 수련시설 수"] / merged_data["10대 인구수"]) * 100000

# 인구 10만 명당 문화·복지·교통·공연·도서관 관련 지표 계산
merged_data["문화시설수(10만명당)"] = (merged_data['문화시설 수'] / merged_data['인구수']) * 100000
merged_data["사회복지시설(10만명당)"] = (merged_data["사회복지 시설 수"] / merged_data["인구수"]) * 100000

merged_data["시내버스이용률(10만명당)"] = (merged_data["시내버스이용"] / merged_data["인구수"]) * 100000
merged_data["지하철이용률(10만명당)"] = (merged_data["지하철이용"] / merged_data["인구수"]) * 100000

merged_data["주1~5회 대중교통 이용률(10만명당)"] = (merged_data["대중교통1~5회이용"] / merged_data["인구수"]) * 100000
merged_data["주6~10회 대중교통 이용률(10만명당)"] = (merged_data["대중교통6~10회이용"] / merged_data["인구수"]) * 100000
merged_data["주11~15회 대중교통 이용률(10만명당)"] = (merged_data["대중교통11~15회이용"] / merged_data["인구수"]) * 100000
merged_data["주16~20회 대중교통 이용률(10만명당)"] = (merged_data["대중교통16~20회이용"] / merged_data["인구수"]) * 100000
merged_data["주20회이상 대중교통 이용률(10만명당)"] = (merged_data["대중교통21회이상이용"] / merged_data["인구수"]) * 100000

merged_data["공연매출금액(10만명당)"] = (merged_data["매출금액"] / merged_data["인구수"]) * 100000
merged_data["공연예매건수(10만명당)"] = (merged_data["예매건수"] / merged_data["인구수"]) * 100000
merged_data["공연시설수(10만명당)"] = (merged_data["시설수"] / merged_data["인구수"]) * 100000
merged_data["공연장수(10만명당)"] = (merged_data["공연장수"] / merged_data["인구수"]) * 100000
merged_data["공개영화수(10만명당)"] = (merged_data["공개영화수"] / merged_data["인구수"]) * 100000
merged_data["궁능관광객수(10만명당)"] = (merged_data["궁능관광객수"] / merged_data["인구수"]) * 100000

merged_data["도서관 수 (10만명당)"] = (merged_data["도서관 수"] / merged_data["인구수"]) * 100000
merged_data["연간대출 도서 수 (10만명당)"] = (merged_data["연간대출 도서 수"] / merged_data["인구수"]) * 100000
merged_data["도서관 방문자 수 (10만명당)"] = (merged_data["도서관 방문자 수"] / merged_data["인구수"]) * 100000


In [29]:
# 모든 수치형 데이터를 소수점 두 자리로 반올림
numerical_columns = ["지정문화재(10만명당)", "국가등록문화재(10만명당)", "청소년수련시설(청소년 10만명당)",
                     "사회복지시설(10만명당)", "문화시설수(10만명당)", "시내버스이용률(10만명당)", "지하철이용률(10만명당)",
                     "주1~5회 대중교통 이용률(10만명당)", "주6~10회 대중교통 이용률(10만명당)", "주11~15회 대중교통 이용률(10만명당)",
                     "주16~20회 대중교통 이용률(10만명당)", "주20회이상 대중교통 이용률(10만명당)", "공연매출금액(10만명당)",
                     "공연예매건수(10만명당)", "공연시설수(10만명당)", "공연장수(10만명당)", "공개영화수(10만명당)", "궁능관광객수(10만명당)","도서관 수 (10만명당)","연간대출 도서 수 (10만명당)", "도서관 방문자 수 (10만명당)", "청소년증발급비율"]

for column in numerical_columns:
    merged_data[column] = merged_data[column].round(2)

In [30]:
# 결과 DataFrame에서도 반올림 적용
result = merged_data[["시도별", "연도", "지정문화재(10만명당)", "국가등록문화재(10만명당)", "청소년수련시설(청소년 10만명당)",
                     "사회복지시설(10만명당)", "문화시설수(10만명당)", "시내버스이용률(10만명당)", "지하철이용률(10만명당)",
                     "주1~5회 대중교통 이용률(10만명당)", "주6~10회 대중교통 이용률(10만명당)", "주11~15회 대중교통 이용률(10만명당)",
                     "주16~20회 대중교통 이용률(10만명당)", "주20회이상 대중교통 이용률(10만명당)",
                     "공연예매건수(10만명당)", "공연시설수(10만명당)", "공연장수(10만명당)", "공개영화수(10만명당)","도서관 수 (10만명당)",
                      "연간대출 도서 수 (10만명당)", "도서관 방문자 수 (10만명당)", "청소년증발급비율"]]

# 결과 저장
result.to_csv('분석 최종 데이터셋.csv', encoding='cp949', index=False)

In [31]:
result

,시도별,연도,지정문화재(10만명당),국가등록문화재(10만명당),청소년수련시설(청소년 10만명당),사회복지시설(10만명당),문화시설수(10만명당),시내버스이용률(10만명당),지하철이용률(10만명당),주1~5회 대중교통 이용률(10만명당),...,주16~20회 대중교통 이용률(10만명당),주20회이상 대중교통 이용률(10만명당),공연예매건수(10만명당),공연시설수(10만명당),공연장수(10만명당),공개영화수(10만명당),도서관 수 (10만명당),연간대출 도서 수 (10만명당),도서관 방문자 수 (10만명당),청소년증발급비율
0,서울,2020,17.44,2.38,2.44,3.25,0.05,0.33,0.71,0.24,...,0.09,0.04,2231.09,5.91,7.62,4.35,1.95,509423.03,207190.45,1.95
1,부산,2020,14.60,0.66,2.55,1.79,0.12,1.68,1.30,0.74,...,0.23,0.10,367.77,2.47,3.22,1.91,1.40,131291.78,169837.49,3.40
2,대구,2020,11.31,0.54,1.31,1.04,0.16,2.37,1.78,1.15,...,0.31,0.10,751.37,2.86,3.85,3.73,1.82,164113.71,153311.12,2.75
3,인천,2020,8.78,0.30,2.58,2.91,0.14,1.72,1.67,0.90,...,0.27,0.11,169.84,1.22,1.66,1.05,1.86,121263.27,153875.23,2.88
4,광주,2020,9.46,1.42,0.63,0.88,0.34,5.65,1.11,2.01,...,0.43,0.18,454.06,2.37,3.45,1.69,1.62,107568.86,137406.51,3.47
5,대전,2020,13.74,1.47,1.28,1.34,0.28,5.13,1.58,2.03,...,0.46,0.21,397.29,3.28,3.82,3.22,1.68,146134.46,128717.80,4.35
6,울산,2020,12.38,0.53,3.65,1.58,0.36,8.78,0.00,2.80,...,0.26,0.03,245.29,1.93,2.55,1.23,1.67,172809.81,236084.12,1.89
7,세종,2020,14.07,0.57,7.17,4.31,1.87,28.71,0.00,10.74,...,0.89,0.20,167.96,2.30,3.16,1.44,3.45,260938.50,141113.17,9.08
8,경기,2020,7.88,0.66,3.39,3.00,0.03,0.46,0.28,0.20,...,0.06,0.03,148.66,1.25,1.79,0.82,2.13,201296.81,189538.86,4.67
9,강원,2020,41.40,2.96,7.98,1.51,1.00,6.46,0.12,2.69,...,0.19,0.07,91.75,3.55,5.27,1.32,3.88,121461.52,203663.55,4.77


In [32]:
import pandas as pd

# -----------------------------------------
# 1) 파일 경로 정의
# -----------------------------------------
file_paths_updated = {
    "지역별 인구수": ('/content/지역별 인구수.csv', 'utf-8'),
    "지정문화재 수": ('/content/지정문화재 수.csv', 'EUC-KR'),
    "국가등록 문화재 수": ('/content/국가등록 문화재 수.csv', 'UTF-8-SIG'),
    "청소년 수련 시설 수": ('/content/청소년 수련 시설 수.csv', 'utf-8'),
    "지역별 10대 인구수": ('/content/지역별 10대 인구수.csv', 'utf-8'),
    "사회 복지 시설 수": ('/content/사회복지시설 수.csv', 'cp949'),
    "문화시설 수": ('/content/인구 대비 문화시설 수.csv', 'cp949'),
    "대중교통이용-시내버스": ("/content/대중교통이용비율(시내버스).csv", 'utf-8'),
    "대중교통이용-지하철": ("/content/대중교통이용비율(지하철).csv", 'utf-8'),
    "대중교통이용 횟수(1~5)": ("/content/대중교통이용횟수(주 1~5).csv", 'utf-8'),
    "대중교통이용 횟수(6~10)": ("/content/대중교통이용횟수(주 6~10).csv", 'utf-8'),
    "대중교통이용 횟수(11~15)": ("/content/대중교통이용횟수(주 11~15).csv", 'utf-8'),
    "대중교통이용 횟수(16~20)": ("/content/대중교통이용횟수(주 16~20).csv", 'utf-8'),
    "대중교통이용 횟수(21~)": ("/content/대중교통이용횟수(주 21~).csv", 'utf-8'),
    "매출금액": ("/content/매출금액_by_year.csv", 'utf-8'),
    "예매건수": ("/content/예매건수_by_year.csv", 'utf-8'),
    "시설수": ("/content/시설수_by_year.csv", 'utf-8'),
    "공연장수": ("/content/공연장수_by_year.csv", 'utf-8'),
    "공개영화수": ("/content/공개영화수_by_year.csv", 'utf-8'),
    "궁능관광객수": ("/content/연도별 궁능 관람객 수.csv", 'cp949'),
    "청소년증발급비율": ("/content/청소년증 발급 비율.csv", 'utf-8'),
    "도서관 수": ('/content/지역별도서관현황.csv', 'cp949'),
    "연간대출 도서 수": ('/content/연간 대출 도서 수.csv', 'cp949'),
    "도서관 방문자 수": ("/content/도서관방문자수.csv", 'cp949')
}

# -----------------------------------------
# 2) 모든 파일 읽기
# -----------------------------------------
loaded = {}
for key, (path, encoding) in file_paths_updated.items():
    loaded[key] = pd.read_csv(path, encoding=encoding)

# -----------------------------------------
# 3) Melt가 필요한 컬럼은 자동 melt
# -----------------------------------------
melt_targets = [
    "지정문화재 수", "국가등록 문화재 수", "청소년 수련 시설 수", "지역별 인구수",
    "지역별 10대 인구수", "사회 복지 시설 수", "문화시설 수",
    "대중교통이용-시내버스", "대중교통이용-지하철",
    "대중교통이용 횟수(1~5)", "대중교통이용 횟수(6~10)",
    "대중교통이용 횟수(11~15)", "대중교통이용 횟수(16~20)",
    "대중교통이용 횟수(21~)", "매출금액", "예매건수", "시설수",
    "공연장수", "공개영화수", "궁능관광객수",
    "도서관 수", "연간대출 도서 수", "도서관 방문자 수"
]

melted = {}
for key in melt_targets:
    df = loaded[key]
    melted[key] = df.melt(id_vars=["시도별"], var_name="연도", value_name=key)

# -----------------------------------------
# 4) 연도(int 변환) → 반복문으로 자동 처리
# -----------------------------------------
for key in melted:
    melted[key]["연도"] = pd.to_numeric(melted[key]["연도"], errors="coerce").astype("Int64")

# 청소년증 발급비율 특별 처리
youth = loaded["청소년증발급비율"]
youth["연도"] = pd.to_numeric(youth["연도"], errors="coerce").astype("Int64")
youth = youth.rename(columns={"청소년증 발급 비율": "청소년증발급비율"})
youth = youth[["시도별", "연도", "청소년증발급비율"]]

# -----------------------------------------
# 5) 병합 – melt된 데이터를 자동으로 모두 merge
# -----------------------------------------
merged = melted["지정문화재 수"]
merge_keys = list(melted.keys())
merge_keys.remove("지정문화재 수")

for key in merge_keys:
    merged = merged.merge(melted[key], on=["시도별", "연도"], how="left")

merged = merged.merge(youth, on=["시도별", "연도"], how="left")

# -----------------------------------------
# 6) 10만 명당 지표 자동 계산
# -----------------------------------------
calc_items = {
    "지정문화재(10만명당)": ("지정문화재 수", "지역별 인구수"),
    "국가등록문화재(10만명당)": ("국가등록 문화재 수", "지역별 인구수"),
    "청소년수련시설(청소년 10만명당)": ("청소년 수련 시설 수", "지역별 10대 인구수"),
    "문화시설수(10만명당)": ("문화시설 수", "지역별 인구수"),
    "사회복지시설(10만명당)": ("사회 복지 시설 수", "지역별 인구수"),
    "시내버스이용률(10만명당)": ("대중교통이용-시내버스", "지역별 인구수"),
    "지하철이용률(10만명당)": ("대중교통이용-지하철", "지역별 인구수"),
    "주1~5회 대중교통 이용률(10만명당)": ("대중교통이용 횟수(1~5)", "지역별 인구수"),
    "주6~10회 대중교통 이용률(10만명당)": ("대중교통이용 횟수(6~10)", "지역별 인구수"),
    "주11~15회 대중교통 이용률(10만명당)": ("대중교통이용 횟수(11~15)", "지역별 인구수"),
    "주16~20회 대중교통 이용률(10만명당)": ("대중교통이용 횟수(16~20)", "지역별 인구수"),
    "주20회이상 대중교통 이용률(10만명당)": ("대중교통이용 횟수(21~)", "지역별 인구수"),
    "공연매출금액(10만명당)": ("매출금액", "지역별 인구수"),
    "공연예매건수(10만명당)": ("예매건수", "지역별 인구수"),
    "공연시설수(10만명당)": ("시설수", "지역별 인구수"),
    "공연장수(10만명당)": ("공연장수", "지역별 인구수"),
    "공개영화수(10만명당)": ("공개영화수", "지역별 인구수"),
    "궁능관광객수(10만명당)": ("궁능관광객수", "지역별 인구수"),
    "도서관 수 (10만명당)": ("도서관 수", "지역별 인구수"),
    "연간대출 도서 수 (10만명당)": ("연간대출 도서 수", "지역별 인구수"),
    "도서관 방문자 수 (10만명당)": ("도서관 방문자 수", "지역별 인구수")
}

for new_col, (col, base) in calc_items.items():
    merged[new_col] = (merged[col] / merged[base]) * 100000

# -----------------------------------------
# 7) 반올림 적용
# -----------------------------------------
for col in calc_items.keys():
    merged[col] = merged[col].round(2)

# -----------------------------------------
# 8) 저장
# -----------------------------------------
merged.to_csv("분석 최종 데이터셋.csv", encoding="cp949", index=False)


In [33]:
# 결과 확인
merged

,시도별,연도,지정문화재 수,국가등록 문화재 수,청소년 수련 시설 수,지역별 인구수,지역별 10대 인구수,사회 복지 시설 수,문화시설 수,대중교통이용-시내버스,...,주20회이상 대중교통 이용률(10만명당),공연매출금액(10만명당),공연예매건수(10만명당),공연시설수(10만명당),공연장수(10만명당),공개영화수(10만명당),궁능관광객수(10만명당),도서관 수 (10만명당),연간대출 도서 수 (10만명당),도서관 방문자 수 (10만명당)
0,서울,2008,1200.0,65.0,12,10111880.0,1230616.0,192.0,2.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,부산,2008,252.0,10.0,3,3514331.0,462837.0,38.0,1.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,대구,2008,183.0,8.0,1,2480984.0,360239.0,15.0,1.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,인천,2008,202.0,5.0,2,2689010.0,384260.0,34.0,1.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,광주,2008,118.0,10.0,1,1465274.0,230488.0,11.0,2.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,전남,2022,1196.0,116.0,13,1775012.0,153006.0,59.0,13.4,100.0,...,0.17,191035.44,7455.78,2.82,4.00,13.24,NaN,4.11,204660.81,278371.19
266,경북,2022,2189.0,66.0,17,2625507.0,222118.0,36.0,8.9,97.6,...,0.12,106955.30,7981.47,2.63,3.73,15.77,NaN,2.70,186353.61,274107.17
267,경남,2022,2290.0,68.0,17,3286961.0,306673.0,65.0,7.2,94.5,...,0.11,241241.80,7714.69,2.13,3.35,15.70,NaN,2.40,269107.03,336296.08
268,제주,2022,369.0,26.0,3,674523.0,67655.0,4.0,19.3,100.0,...,0.46,423310.99,19050.94,4.89,6.08,27.28,NaN,3.26,346506.05,250784.33


## 5. 시도별 데이터 분리

위에서 만들어낸 분석 최종 데이터셋.csv를 통해 시도별로 데이터를 각각 추출해냄.

In [34]:
# CSV 파일 불러오기
file_path = '/content/분석 최종 데이터셋.csv'
data = pd.read_csv(file_path, encoding='cp949')

In [35]:
# '시도별' 컬럼을 기준으로 데이터를 그룹화하고 각 그룹을 별도의 CSV 파일로 저장
for city, group in data.groupby('시도별'):
    # 파일 이름 설정 (시도 이름을 파일명에 포함)
    file_name = f"{city}_data.csv"
    # CSV 파일로 저장
    group.to_csv(file_name, index=False, encoding='utf-8')

    print(f"{city} 데이터가 {file_name} 파일로 저장되었습니다.")

강원 데이터가 강원_data.csv 파일로 저장되었습니다.
경기 데이터가 경기_data.csv 파일로 저장되었습니다.
경남 데이터가 경남_data.csv 파일로 저장되었습니다.
경북 데이터가 경북_data.csv 파일로 저장되었습니다.
광주 데이터가 광주_data.csv 파일로 저장되었습니다.
대구 데이터가 대구_data.csv 파일로 저장되었습니다.
대전 데이터가 대전_data.csv 파일로 저장되었습니다.
부산 데이터가 부산_data.csv 파일로 저장되었습니다.
서울 데이터가 서울_data.csv 파일로 저장되었습니다.
세종 데이터가 세종_data.csv 파일로 저장되었습니다.
울산 데이터가 울산_data.csv 파일로 저장되었습니다.
인천 데이터가 인천_data.csv 파일로 저장되었습니다.
전국 데이터가 전국_data.csv 파일로 저장되었습니다.
전남 데이터가 전남_data.csv 파일로 저장되었습니다.
전북 데이터가 전북_data.csv 파일로 저장되었습니다.
제주 데이터가 제주_data.csv 파일로 저장되었습니다.
충남 데이터가 충남_data.csv 파일로 저장되었습니다.
충북 데이터가 충북_data.csv 파일로 저장되었습니다.
